# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
pip install pyproj

  Using cached pyproj-3.6.0-cp311-cp311-win_amd64.whl (5.7 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
pip install cartopy

                                              0.0/10.9 MB ? eta -:--:--
                                              0.1/10.9 MB 2.3 MB/s eta 0:00:05
                                              0.2/10.9 MB 3.1 MB/s eta 0:00:04
     -                                        0.3/10.9 MB 3.5 MB/s eta 0:00:04
     --                                       0.6/10.9 MB 3.9 MB/s eta 0:00:03
     --                                       0.7/10.9 MB 3.5 MB/s eta 0:00:03
     ---                                      0.9/10.9 MB 4.0 MB/s eta 0:00:03
     ---                                      0.9/10.9 MB 4.0 MB/s eta 0:00:03
     ---                                      0.9/10.9 MB 4.0 MB/s eta 0:00:03
     ---                                      1.0/10.9 MB 3.1 MB/s eta 0:00:04
     ----                                     1.3/10.9 MB 3.5 MB/s eta 0:00:03
     -----                                    1.5/10.9 MB 3.9 MB/s eta 0:00:03
     ------                                   1.8/10.9 MB 4

  error: subprocess-exited-with-error
  
  × Building wheel for cartopy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [279 lines of output]
      <string>:90: UserWarning: Unable to determine GEOS version. Ensure you have 3.7.2 or later installed, or installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\crs.py -> build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\geodesic.py -> build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\img_transform.py -> build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\util.py -> build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\vector_transform.py -> build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\_epsg.py -> build\lib.win-amd64-cpython-311\cartopy
      copying lib\cartopy\_

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(x="Lng", y="Lat", frame_height=450, frame_width=800, tiles="OSM", color="City", size="Humidity", alpha = 0.5)

# Display the map
city_map


ImportError: Geographic projection support requires: cartopy, geoviews.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
indexhumid = city_data_df[(city_data_df['Humidity'] < 45)].index
city_data_df.drop(indexhumid, inplace=True)
indextemp = city_data_df[(city_data_df['Max Temp'] < 25) & (city_data_df['Max Temp'] > 47)].index
city_data_df.drop(indextemp, inplace=True)
indexwind = city_data_df[(city_data_df['Wind Speed'] > 6.5)].index
city_data_df.drop(indexwind, inplace=True)
indexcloud = city_data_df[(city_data_df['Cloudiness'] > 25)].index
city_data_df.drop(indexcloud, inplace=True)

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
33,33,stornoway,58.2093,-6.3865,12.18,79,20,3.09,GB,1666108251
42,42,margate,51.3813,1.3862,16.84,76,3,1.34,GB,1666108256
43,43,ancud,-41.8697,-73.8203,14.75,70,11,2.68,CL,1666108257


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head(20)


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,
20,avarua,CK,-21.2078,-159.7750,73,
33,stornoway,GB,58.2093,-6.3865,79,
42,margate,GB,51.3813,1.3862,76,
43,ancud,CL,-41.8697,-73.8203,70,
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
53,lasa,IT,46.6166,10.7002,68,
54,vaini,TO,-21.2000,-175.2000,100,
59,busselton,AU,-33.6500,115.3333,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


In [36]:

# Set parameters to search for a hotel
radius = 10000

params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

# Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



saint-pierre - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
stornoway - nearest hotel: No hotel found
margate - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
lasa - nearest hotel: No hotel found
vaini - nearest hotel: No hotel found
busselton - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
esperance - nearest hotel: No hotel found
lichuan - nearest hotel: No hotel found
jashpurnagar - nearest hotel: No hotel found
half moon bay - nearest hotel: No hotel found
ambon - nearest hotel: No hotel found
corowa - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
tawang - nearest hotel: No hotel found
puerto del rosario - nearest hotel: No hotel found
mount gambier - nearest hotel: No hotel found
lavrentiya - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
maragogi - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
20,avarua,CK,-21.2078,-159.7750,73,No hotel found
33,stornoway,GB,58.2093,-6.3865,79,No hotel found
42,margate,GB,51.3813,1.3862,76,No hotel found
43,ancud,CL,-41.8697,-73.8203,70,No hotel found
...,...,...,...,...,...,...
559,lompoc,US,34.6391,-120.4579,100,No hotel found
564,redmond,US,47.6740,-122.1215,91,No hotel found
571,dargaville,NZ,-35.9333,173.8833,83,No hotel found
575,iquique,CL,-20.2208,-70.1431,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

ImportError: Geographic projection support requires: cartopy, geoviews.